# Contextual Embeddings and Semantic Search Engines with Transformers

# Finding semantic similarity with Pre-trained Embeddings

Here we will leverage already pre-trained embedding models \ deep learning models to extract embeddings from sentences and find out their semantic similarity.

Models we will look at:

- Transformers

# Semantic Search

Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines which only find documents based on lexical matches, semantic search can also find synonyms.


The idea behind semantic search is to embed all entries in your corpus, whether they be sentences, paragraphs, or documents, into a vector space.

At search time, the query is embedded into the same vector space and the closest embeddings from your corpus are found. These entries should have a high semantic overlap with the query.

![](https://i.imgur.com/FyUCkG5.png)


# Fun with Embeddings: Simple Search Engine!

Let's create a corpus of documents which will be our source on which we will run text searches

In [1]:
documents = [
  "Quantum mechanics describes the behavior of very small particles.",
  "Photosynthesis is the process by which green plants make food using sunlight.",
  "Shakespeare's plays are a testament to English literature.",
  "Artificial Intelligence aims to create machines that can think and learn.",
  "The pyramids of Egypt are historical monuments that have stood for thousands of years.",
  "Biology is the study of living organisms and their interactions with the environment.",
  "Music therapy can aid in the mental well-being of individuals.",
  "The Milky Way is just one of billions of galaxies in the universe.",
  "Economic theories help understand the distribution of resources in society.",
  "Yoga is an ancient practice that involves physical postures and meditation."
]

## Building Robust Semantic Search Engines with Transformers

In [2]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [ ]:
!nvidia-smi

Thu Sep 12 22:57:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load Dependencies

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch

## Load Pre-trained Transformer Model

In [4]:
# https://huggingface.co/microsoft/MiniLM-L12-H384-uncased
# MiniLM: Small and Fast Pre-trained Models for Language Understanding and Generation
# MiniLMv1-L12-H384-uncased: 12-layer, 384-hidden, 12-heads, 33M parameters, 2.7x faster than BERT-Base

model = SentenceTransformer('all-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create a corpus of documents

In [5]:
documents = ['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Yoga is an ancient practice that involves physical postures and meditation.']

In [ ]:
len(documents)

10

## Get document embeddings

In [6]:
document_embeddings = model.encode(documents)

In [7]:
document_embeddings, document_embeddings.shape

(array([[ 0.01569205,  0.02793592,  0.01044163, ...,  0.05586382,
         -0.06863529,  0.04651124],
        [-0.03118527,  0.03116489,  0.01921587, ..., -0.03559601,
          0.0487383 ,  0.0257752 ],
        [ 0.01772631, -0.03875763,  0.12742487, ...,  0.06566847,
          0.05715033, -0.05221871],
        ...,
        [ 0.09988892, -0.04224407,  0.04059114, ..., -0.05507799,
          0.03537545, -0.07602306],
        [-0.01405635, -0.00320508, -0.01274013, ...,  0.01621084,
          0.0084957 ,  0.00209839],
        [-0.04375399, -0.00201995, -0.00192908, ...,  0.08587868,
         -0.00045801, -0.02675951]], dtype=float32),
 (10, 384))

## Let's try to find the most similar document for one query

### New Query

In [8]:
new_text = 'What is AI?'
new_text

'What is AI?'

### Get Embedding for New Query

In [9]:
new_text_embedding = model.encode(new_text)
new_text_embedding.shape

(384,)

### Get Cosine Similarity Score of Document Emebddings compared to New Query Embedding

In [10]:
cos_scores = util.cos_sim(new_text_embedding, document_embeddings)[0]
cos_scores

tensor([ 5.4058e-02, -4.1397e-02,  6.0208e-02,  6.1713e-01,  4.6178e-04,
         1.1132e-01,  6.8011e-02, -4.1013e-02,  1.3222e-01,  6.8518e-02])

### Get Most Similar Document ID

In [11]:
top_results = torch.topk(cos_scores, k=1)
top_results

torch.return_types.topk(
values=tensor([0.6171]),
indices=tensor([3]))

In [12]:
idx = top_results.indices.item()
idx

3

### Get Most Similar Document

In [13]:
documents[idx]

'Artificial Intelligence aims to create machines that can think and learn.'

## Create a function to return the top similar document based on any query

In [14]:
def semantic_search_engine(query, embedder_model):

  query_embedding = model.encode(query)
  # We use cosine-similarity and torch.topk to find the highest similarity score
  cos_scores = util.cos_sim(query_embedding, document_embeddings)[0]
  top_results = torch.topk(cos_scores, k=1)
  return documents[top_results.indices.item()]

## Try out the function

In [15]:
new_sentence = 'Tell me about AI'
semantic_search_engine(new_sentence, model)

'Artificial Intelligence aims to create machines that can think and learn.'

In [16]:
new_sentence = 'Do you know about the pyramids?'
semantic_search_engine(new_sentence, model)

'The pyramids of Egypt are historical monuments that have stood for thousands of years.'

In [17]:
new_sentence = 'How do plants survive?'
semantic_search_engine(new_sentence, model)

'Photosynthesis is the process by which green plants make food using sunlight.'